In [1]:
import math
import random
import os
import time
import json
from functools import partial
from collections import defaultdict

import numpy as np
import pandas as pd

%matplotlib notebook
import matplotlib
import matplotlib.pyplot as plt

import tensorflow as tf

### Check Version

In [2]:
tf.__version__

'2.9.1'

### GPU Configuration

In [3]:
gpus = tf.config.list_physical_devices('GPU')

2022-11-10 01:35:31.100968: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-10 01:35:31.107692: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-10 01:35:31.108130: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [4]:
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
tf.config.experimental.set_memory_growth(gpus[0], True)

### Pandas Config

In [6]:
pd.set_option('display.max_column', None)

### Input Pipeline

In [7]:
IMG_WIDTH = 256
IMG_HEIGHT = 256

TRAIN_BATCH_SIZE = 4
VAL_BATCH_SIZE = 4

In [8]:
CROP_WIDTH = 227
CROP_HEIGHT = 227

In [9]:
foldFiles = ["adience/unprocessed/fold_0_data.txt",
             "adience/unprocessed/fold_1_data.txt",
             "adience/unprocessed/fold_2_data.txt",
             "adience/unprocessed/fold_3_data.txt",
             "adience/unprocessed/fold_4_data.txt"]

In [10]:
genderMap = defaultdict(lambda : np.NaN)
genderMap['m'] = 0
genderMap['f'] = 1

In [11]:
ages = ['(0, 2)', '(4, 6)', '(8, 13)', '(15, 20)', '(25, 32)', '(38, 43)', '(48, 53)', '(60, 100)']
ageMap = defaultdict(lambda : np.NaN)
for i,a in enumerate(ages):
    ageMap[a] = i

### Dataset Generation

Dataset generation pipeline:
Input: 
- foldFile - Path to fold file. Fold file Describes which images are in the fold and its corresponding labels
- imgBaseFolder - Base folder to search image from
- imgPrefix - Prefix of image file
- genderMap - Map from ['m', 'f', 'u', None], to one hot index
- ageMap - Map from age category to one hot index
- imgWidth - Resulting image width
- imgHeigh - Resulting image height
- batchSize - Int or None, batch size
- configureDs - Function accepting dataset for performance configurations
- preBatch - List of (name, functions) pair that will be mapped before batching. name is used as name parameters for tf graph 
- postBatch - List of (name, functions) that will be mapped after batching. name is used as name parameters for tf graph

The processing functions should have signature function(img, label) -> (img, label)

Pipeline

Read Fold File -> Preprocess filename and labels (dataframe) -> Convert filename and labels to numpy array -> Convert filename and labels to tf dataset -> Parse images and labels -> Configure Dataset for performance -> Pre-Batching preprocessing -> Batch -> Post-Batching preprocessing -> Output

Some preprocessing steps can only be done before and some can only be done after batching, thats why there are seperated pre and post batching list

In [12]:
def generateDs(foldFiles, imgBaseFolder, imgPrefix, genderMap, ageMap, genderDepth, ageDepth, imgWidth, imgHeight, batchSize, configureDs=None, preBatch=[], postBatch=[]):
    
    def parseImage(filename):
        img = tf.io.read_file(filename)
        img = tf.image.decode_jpeg(img)
        img = tf.image.resize(img, [imgHeight, imgWidth])
         
        return img
    
    # Read Fold File
    foldData = []
    for f in foldFiles:
        foldData.append(pd.read_csv(f, sep="\t"))
    foldData = pd.concat(foldData)
    
    # Form File Name
    foldData['filename'] = foldData.apply(lambda r: os.path.join(imgBaseFolder, r['user_id'], f"{imgPrefix}.{r['face_id']}.{r['original_image']}"), axis=1)
    
    # Generate Label One Hot Index
    foldData['gender_ind'] = foldData['gender'].map(genderMap)
    foldData['age_ind'] = foldData['age'].map(ageMap)
    
    # Remove dirty data
    foldData.dropna(subset=['gender_ind', 'age_ind'], inplace=True)
    
    # Dataframe to numpy
    filenames = foldData['filename'].to_numpy()
    
    genderIndex = foldData['gender_ind'].to_numpy().astype(int)
    ageIndex = foldData['age_ind'].to_numpy().astype(int)
    
    # Numpy to Dataset
    fnDs = tf.data.Dataset.from_tensor_slices(filenames)
    genderIndDs = tf.data.Dataset.from_tensor_slices(genderIndex)
    ageIndDs = tf.data.Dataset.from_tensor_slices(ageIndex)
    
    # Parse Images
    imageDs = fnDs.map(parseImage, num_parallel_calls=tf.data.AUTOTUNE, name="parse_image")
    
    # Parse Labels
    genderLabDs = genderIndDs.map(lambda x: tf.one_hot(x, genderDepth), num_parallel_calls=tf.data.AUTOTUNE, name="gender_one_hot")
    ageLabDs = ageIndDs.map(lambda x: tf.one_hot(x, ageDepth), num_parallel_calls=tf.data.AUTOTUNE, name="age_one_hot")
    
    # Combine Labels
    labelDs = tf.data.Dataset.zip((genderLabDs, ageLabDs), name="label_zip")
    labelDs = labelDs.map(lambda g,a: {"gender": g, "age": a}, num_parallel_calls=tf.data.AUTOTUNE, name='label_dict')
    
    # Combine Images and Labels into dataset
    ds = tf.data.Dataset.zip((imageDs, labelDs))
    
    # Configure Performance
    if(configureDs is not None):
        ds = configureDs(ds)
    
    # Pre Batch Preprocessing
    for n,f in preBatch:
        ds = ds.map(f, num_parallel_calls=tf.data.AUTOTUNE, name=n)
    
    # Batch
    if(batchSize is not None):
        ds = ds.batch(batchSize, name="ds_batch")
    
    # Post Batch Preprocessing
    for n,f in postBatch:
        ds = ds.map(f, num_parallel_calls=tf.data.AUTOTUNE, name=n)
    
    return ds

### Preprocessings

In [13]:
def trainConfigPerformance(ds):
    #ds = ds.cache()
    ds = ds.shuffle(buffer_size=100)
    ds = ds.repeat()
    return ds

In [14]:
def valConfigPerformance(ds):
    #ds = ds.cache()
    return ds

#### Preprocessing steps according to the reference paper

In [15]:
def trainPreprocessA(img, lab):
        
        # img = tf.image.random_crop(img, [CROP_HEIGHT, CROP_WIDTH, 3])
        
        ud = tf.random.uniform([], dtype=tf.float32)
        udCond = tf.less(ud, 0.5)
        
        img = tf.cond(udCond, lambda: tf.image.flip_up_down(img), lambda: img)
        
        lr = tf.random.uniform([], dtype=tf.float32)
        lrCond = tf.less(lr, 0.5)
        
        img = tf.cond(lrCond, lambda: tf.image.flip_left_right(img), lambda: img)
        
        img = tf.image.random_brightness(img, 63/255)
        
        img = tf.image.random_contrast(img, 0.2, 1.8) 
        
        return img, lab

In [16]:
def valPreprocessA(img, lab):
        
        img = tf.image.crop_to_bounding_box(img,
                                            (IMG_HEIGHT-CROP_HEIGHT) // 2,
                                            (IMG_WIDTH-CROP_WIDTH) // 2,
                                            CROP_HEIGHT,
                                            CROP_WIDTH)
        
        return img, lab

In [17]:
def imageStandardize(img, lab):
    
    img = tf.image.per_image_standardization(img)
    
    return img, lab

#### Label extractor 

In [18]:
def extractGenderLabel(img, lab):
    
    lab = lab['gender']
    
    return img, lab

In [19]:
def extractAgeLabel(img, lab):
    
    lab = lab['age']
    
    return img, lab

### Generate Folds for K-Folds validation

In [20]:
def generateFoldDs(foldFiles, imgBaseFolder, imgPrefix, genderMap, ageMap, genderDepth, ageDepth, imgWidth, imgHeight, trainBatchSize, valBatchSize, trainConfigureDs=None, trainPreBatch=[], trainPostBatch=[], valConfigureDs=None, valPreBatch=[], valPostBatch=[]):
    '''
        Returns list of (train, validation) datasets
    '''
    
    N = len(foldFiles)
    
    folds = []
    
    for i in range(N):
        
        trainFiles = foldFiles[:i]
        if(i < N-1):
            trainFiles.extend(foldFiles[i+1:])
            
        valFiles = foldFiles[i]
        
        trainDs = generateDs(foldFiles, imgBaseFolder, imgPrefix, genderMap, ageMap, genderDepth, ageDepth, imgWidth, imgHeight, trainBatchSize, configureDs=trainConfigureDs, preBatch=trainPreBatch, postBatch=trainPostBatch)
        valDs = generateDs(foldFiles, imgBaseFolder, imgPrefix, genderMap, ageMap, genderDepth, ageDepth, imgWidth, imgHeight, valBatchSize, configureDs=valConfigureDs, preBatch=valPreBatch, postBatch=valPostBatch)
        
        folds.append((trainDs, valDs))
        
    return folds

In [21]:
trainPreBatch = [("train_process_a", trainPreprocessA),
                 ("train_extract_gender", extractGenderLabel)]

valPreBatch = [("val_extract_gender", extractGenderLabel)]

In [22]:
folds = generateFoldDs(foldFiles, "adience/faces", "coarse_tilt_aligned_face", genderMap, ageMap, 2, 8, 
                       IMG_WIDTH, IMG_HEIGHT, TRAIN_BATCH_SIZE, VAL_BATCH_SIZE, 
                       trainConfigureDs=trainConfigPerformance, trainPreBatch=trainPreBatch, 
                       valConfigureDs=valConfigPerformance, valPreBatch=valPreBatch)

2022-11-10 01:35:31.443390: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-10 01:35:31.444072: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-10 01:35:31.444329: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-10 01:35:31.444526: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [23]:
len(folds)

5

### Check data

In [24]:
for x,y in folds[0][0].take(1):
    pass

In [25]:
x

<tf.Tensor: shape=(4, 256, 256, 3), dtype=float32, numpy=
array([[[[ 5.87747192e+00,  2.41669907e+02,  2.61731628e+02],
         [ 3.31987000e+00,  2.39112305e+02,  2.59174011e+02],
         [ 1.61071777e-01,  2.35953506e+02,  2.56015198e+02],
         ...,
         [ 3.91357040e+01,  1.08798981e+02,  1.98165924e+02],
         [-1.07546082e+01,  5.89683533e+01,  1.45663498e+02],
         [-6.95587158e+00,  6.53587799e+01,  1.46417648e+02]],

        [[ 1.43446350e+00,  2.37226913e+02,  2.57288635e+02],
         [-2.73725891e+00,  2.33055176e+02,  2.53116882e+02],
         [ 1.53663635e+00,  2.36556305e+02,  2.57004456e+02],
         ...,
         [ 8.98303986e+00,  6.69413147e+01,  1.65046021e+02],
         [-4.78161469e+01,  9.13439941e+00,  1.03128006e+02],
         [-2.34003448e+01,  3.64581604e+01,  1.26073257e+02]],

        [[ 4.03094482e+00,  2.38249741e+02,  2.58311462e+02],
         [-2.70399475e+00,  2.31514832e+02,  2.51576538e+02],
         [-5.85159302e+00,  2.27466171e+02

In [26]:
y

<tf.Tensor: shape=(4, 2), dtype=float32, numpy=
array([[0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.]], dtype=float32)>

### Model

In [27]:
def createModel():
    inp = tf.keras.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))

    conv1 = tf.keras.layers.Conv2D(96, 7, strides=(4,4), activation='relu', kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01), name='conv1')
    pool1 = tf.keras.layers.MaxPool2D(pool_size=(3,3), strides=2, name='pool1')

    conv2 = tf.keras.layers.Conv2D(256, 5, activation='relu', kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01), name='conv2')
    pool2 = tf.keras.layers.MaxPool2D(pool_size=(3,3), strides=2, name='pool2')

    conv3 = tf.keras.layers.Conv2D(384, 3, activation='relu', kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01), name='conv3')
    pool3 = tf.keras.layers.MaxPool2D(pool_size=(3,3), strides=2, name='pool3')

    flat1 = tf.keras.layers.Flatten(name='flat1')
    fc1 = tf.keras.layers.Dense(512, activation='relu', kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.005), bias_initializer='ones', name='fc1')
    do1 = tf.keras.layers.Dropout(0.5, name='do1')

    fc2 = tf.keras.layers.Dense(512, activation='relu', kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.005), bias_initializer='ones', name='fc2')
    do2 = tf.keras.layers.Dropout(0.5, name='do2')

    fc3 = tf.keras.layers.Dense(2, activation='softmax', kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01), name='fc3')

    o = (inp / 127.5) - 1.0
    o = conv1(o)
    o = pool1(o)
    o = tf.nn.local_response_normalization(o, depth_radius=5, alpha=0.0001, beta=0.75)

    o = tf.pad(o, [[0, 0], [2, 2], [2, 2], [0, 0]])
    o = conv2(o)
    o = pool2(o)
    o = tf.nn.local_response_normalization(o, depth_radius=5, alpha=0.0001, beta=0.75)

    o = tf.pad(o, [[0, 0], [1, 1], [1, 1], [0, 0]])
    o = conv3(o)
    o = pool3(o)
    o = tf.nn.local_response_normalization(o, depth_radius=5, alpha=0.0001, beta=0.75)

    o = flat1(o)
    o = fc1(o)
    o = do1(o)


    o = fc2(o)
    o = do2(o)

    o = fc3(o)

    model = tf.keras.models.Model(inputs=inp, outputs=o, name='cnn_8')
    
    return model

### Compile Model

In [28]:
STEPS_PER_EPOCH = 1000

EPOCH = 300

START_EPOCH = 0
END_EPOCH = START_EPOCH + EPOCH

#### Learning Rate Schedule

In [29]:
def schedule(epoch, lr):
    if(epoch == 10000*50//STEPS_PER_EPOCH):
        return lr * 0.1
    else:
        return lr

In [30]:
logBasePath = "log/CNN_8"
logPrefix = "log"

In [31]:
for i, (train, val) in enumerate([folds[0]]):
    
    model = createModel()
    
    opt = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)
    loss = tf.keras.losses.CategoricalCrossentropy()
    metrics = ['accuracy']
    
    model.compile(optimizer=opt, loss=loss, metrics=metrics)
    
    tboardCb = tf.keras.callbacks.TensorBoard(log_dir=f"{logBasePath}/{logPrefix}_{i}", histogram_freq=1, profile_batch=(2,10))
    lrSchedule = tf.keras.callbacks.LearningRateScheduler(schedule)
    
    cbs = [tboardCb]
    
    model.evaluate(val)
    model.fit(train, initial_epoch=START_EPOCH, epochs=END_EPOCH, steps_per_epoch=STEPS_PER_EPOCH, callbacks=cbs, validation_data=val)

2022-11-10 01:35:33.855294: I tensorflow/core/profiler/lib/profiler_session.cc:99] Profiler session initializing.
2022-11-10 01:35:33.855311: I tensorflow/core/profiler/lib/profiler_session.cc:114] Profiler session started.
2022-11-10 01:35:33.855330: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1665] Profiler found 1 GPUs
2022-11-10 01:35:33.933521: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session tear down.
2022-11-10 01:35:33.934733: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1799] CUPTI activity buffer flushed
2022-11-10 01:35:34.453731: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8101


  56/3528 [..............................] - ETA: 9s - loss: 0.6638 - accuracy: 0.6205   

2022-11-10 01:35:35.239870: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 822.72MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2022-11-10 01:35:35.239979: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 822.72MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2022-11-10 01:35:35.248481: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 843.88MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2022-11-10 01:35:35.248497: W tensorflow/core/common_runtime/bfc_al

3528/3528 [==============================] - 11s 3ms/step - loss: 0.7368 - accuracy: 0.4768
Epoch 1/300


2022-11-10 01:35:44.519815: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 819.62MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2022-11-10 01:35:44.519859: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 819.62MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2022-11-10 01:35:44.527552: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 841.75MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2022-11-10 01:35:44.527576: W tensorflow/core/common_runtime/bfc_al

...

1000/1000 [==============================] - 14s 14ms/step - loss: 0.6907 - accuracy: 0.5220 - val_loss: 0.6956 - val_accuracy: 0.4768
Epoch 58/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.6954 - accuracy: 0.4990 - val_loss: 0.6936 - val_accuracy: 0.4768
Epoch 59/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.6728 - accuracy: 0.5920 - val_loss: 0.6926 - val_accuracy: 0.5232
Epoch 60/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.6832 - accuracy: 0.5530 - val_loss: 0.6990 - val_accuracy: 0.5232
Epoch 61/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.6979 - accuracy: 0.4815 - val_loss: 0.6933 - val_accuracy: 0.4768
Epoch 62/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.6917 - accuracy: 0.5337 - val_loss: 0.6922 - val_accuracy: 0.5232
Epoch 63/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.6776 - accuracy: 0.5658 - val_loss: 0.6954 

1000/1000 [==============================] - 14s 15ms/step - loss: 0.6769 - accuracy: 0.5755 - val_loss: 0.6942 - val_accuracy: 0.5232
Epoch 113/300
1000/1000 [==============================] - 15s 15ms/step - loss: 0.6902 - accuracy: 0.5282 - val_loss: 0.6936 - val_accuracy: 0.4768
Epoch 114/300
1000/1000 [==============================] - 15s 15ms/step - loss: 0.6965 - accuracy: 0.4935 - val_loss: 0.6964 - val_accuracy: 0.4768
Epoch 115/300
1000/1000 [==============================] - 15s 15ms/step - loss: 0.6858 - accuracy: 0.5470 - val_loss: 0.6932 - val_accuracy: 0.5232
Epoch 116/300
1000/1000 [==============================] - 15s 15ms/step - loss: 0.6823 - accuracy: 0.5567 - val_loss: 0.6923 - val_accuracy: 0.5232
Epoch 117/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.6909 - accuracy: 0.5285 - val_loss: 0.6976 - val_accuracy: 0.4768
Epoch 118/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.6948 - accuracy: 0.4955 - val_loss: 0

1000/1000 [==============================] - 15s 15ms/step - loss: 0.6968 - accuracy: 0.5090 - val_loss: 0.6921 - val_accuracy: 0.5406
Epoch 168/300
1000/1000 [==============================] - 15s 15ms/step - loss: 0.6576 - accuracy: 0.6127 - val_loss: 0.8246 - val_accuracy: 0.5232
Epoch 169/300
1000/1000 [==============================] - 15s 15ms/step - loss: 0.6572 - accuracy: 0.5800 - val_loss: 0.6894 - val_accuracy: 0.5297
Epoch 170/300
1000/1000 [==============================] - 15s 15ms/step - loss: 0.6529 - accuracy: 0.6292 - val_loss: 0.6701 - val_accuracy: 0.6124
Epoch 171/300
1000/1000 [==============================] - 15s 15ms/step - loss: 0.6609 - accuracy: 0.6003 - val_loss: 0.8112 - val_accuracy: 0.4768
Epoch 172/300
1000/1000 [==============================] - 15s 15ms/step - loss: 0.6364 - accuracy: 0.6360 - val_loss: 0.6647 - val_accuracy: 0.6026
Epoch 173/300
1000/1000 [==============================] - 15s 15ms/step - loss: 0.6527 - accuracy: 0.6168 - val_loss: 0

1000/1000 [==============================] - 15s 15ms/step - loss: 0.4107 - accuracy: 0.8100 - val_loss: 0.3670 - val_accuracy: 0.8384
Epoch 223/300
1000/1000 [==============================] - 14s 15ms/step - loss: 0.3182 - accuracy: 0.8685 - val_loss: 0.5334 - val_accuracy: 0.7499
Epoch 224/300
1000/1000 [==============================] - 15s 15ms/step - loss: 0.3455 - accuracy: 0.8520 - val_loss: 0.7743 - val_accuracy: 0.7188
Epoch 225/300
1000/1000 [==============================] - 15s 15ms/step - loss: 0.3915 - accuracy: 0.8220 - val_loss: 0.3514 - val_accuracy: 0.8440
Epoch 226/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.3357 - accuracy: 0.8447 - val_loss: 0.5676 - val_accuracy: 0.8033
Epoch 227/300
1000/1000 [==============================] - 15s 15ms/step - loss: 0.3265 - accuracy: 0.8637 - val_loss: 0.4422 - val_accuracy: 0.7934
Epoch 228/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.3549 - accuracy: 0.8428 - val_loss: 0

1000/1000 [==============================] - 15s 15ms/step - loss: 0.2244 - accuracy: 0.9090 - val_loss: 0.2653 - val_accuracy: 0.8864
Epoch 278/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.2748 - accuracy: 0.8830 - val_loss: 0.3430 - val_accuracy: 0.8481
Epoch 279/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.2073 - accuracy: 0.9158 - val_loss: 0.3684 - val_accuracy: 0.8508
Epoch 280/300
1000/1000 [==============================] - 15s 15ms/step - loss: 0.2212 - accuracy: 0.9128 - val_loss: 0.3333 - val_accuracy: 0.8644
Epoch 281/300
1000/1000 [==============================] - 15s 15ms/step - loss: 0.2357 - accuracy: 0.9050 - val_loss: 0.2658 - val_accuracy: 0.8964
Epoch 282/300
1000/1000 [==============================] - 15s 15ms/step - loss: 0.2573 - accuracy: 0.8932 - val_loss: 0.2570 - val_accuracy: 0.9040
Epoch 283/300
1000/1000 [==============================] - 15s 15ms/step - loss: 0.1822 - accuracy: 0.9280 - val_loss: 0